In [65]:
import pandas as pd
import glob
import os
from datetime import datetime
import calendar

In [2]:
# df = pd.read_csv("data/IntervalMeterUsage202512302103505093F4F9339E4024C89B21685CA1223B8AEEML.CSV")

In [37]:
# Define the folder path (adjust as needed)
folder_path = './data/'  # e.g., './data/', or 'C:/my_folder/'
# Use glob to find all .csv files
csv_files = glob.glob(os.path.join(folder_path, "*.CSV"))
# Check if any files found
if not csv_files:
    print("No CSV files found.")
else:
    # Read each CSV file and store in a list
    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            dfs.append(df)
            print(f"Loaded: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    # Concatenate all DataFrames
    df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows after combining: {len(df)}")
df = df.drop(columns=["ESTIMATED_ACTUAL","REVISION_DATE", "CONSUMPTION_SURPLUSGENERATION"])
df['USAGE_START_TIME'] = df['USAGE_START_TIME'].str.strip()
df['USAGE_END_TIME'] = df['USAGE_END_TIME'].str.strip()
df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format='%H:%M').dt.hour

# Convert USAGE_DATE to datetime
df['datetime'] = pd.to_datetime(df['USAGE_DATE'], format='%m/%d/%Y')
# Create 15-min interval offsets (0, 15, 30, 45 minutes)
# Assuming rows are ordered chronologically
df['datetime'] += pd.to_timedelta(df['hour'], unit='hour')
# Now you have full timestamps
# print(df[['USAGE_DATE', 'USAGE_KWH', 'datetime']])


Loaded: ./data/IntervalMeterUsage202512232103511433F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512132104342333F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512092104337143F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512062104465213F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512262104295393F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512292104025003F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512242103557593F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512112104178453F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512252104042683F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512072104407293F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512032104445033F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./

In [38]:
df

,ESIID,USAGE_DATE,USAGE_START_TIME,USAGE_END_TIME,USAGE_KWH,hour,datetime
0,'10443720001617272,12/22/2025,00:00,00:15,0.307,0,2025-12-22 00:00:00
1,'10443720001617272,12/22/2025,00:15,00:30,0.293,0,2025-12-22 00:00:00
2,'10443720001617272,12/22/2025,00:30,00:45,0.311,0,2025-12-22 00:00:00
3,'10443720001617272,12/22/2025,00:45,01:00,0.313,0,2025-12-22 00:00:00
4,'10443720001617272,12/22/2025,01:00,01:15,0.287,1,2025-12-22 01:00:00
...,...,...,...,...,...,...,...
2875,'10443720001617272,12/26/2025,22:45,23:00,0.808,22,2025-12-26 22:00:00
2876,'10443720001617272,12/26/2025,23:00,23:15,0.538,23,2025-12-26 23:00:00
2877,'10443720001617272,12/26/2025,23:15,23:30,0.682,23,2025-12-26 23:00:00
2878,'10443720001617272,12/26/2025,23:30,23:45,0.422,23,2025-12-26 23:00:00


In [39]:
# Set datetime as index for resampling
df.set_index('datetime', inplace=True)
# Resample and sum kWh
hourly = df['USAGE_KWH'].resample('h').sum()
daily = df['USAGE_KWH'].resample('D').sum()
weekly = df['USAGE_KWH'].resample('W').sum()
monthly = df['USAGE_KWH'].resample('ME').sum()
# date =df['USAGE_DATE']

In [40]:
summary_df = pd.DataFrame({
    'hourly_kwh': hourly,
    'daily_kwh': daily,
    'weekly_kwh': weekly,
    'monthly_kwh': monthly
}).reset_index()
# Rename index to 'datetime'
summary_df.rename(columns={'index': 'datetime'}, inplace=True)

# Extract frequency labels from datetime
df.reset_index(inplace=True)
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.date
df['week'] = df['datetime'].dt.to_period('W')
df['month'] = df['datetime'].dt.to_period('M')
# Group by hour/day/week/month and aggregate
hourly_agg = df.groupby('hour')['USAGE_KWH'].sum().rename('hourly_kwh')
daily_agg = df.groupby('day')['USAGE_KWH'].sum().rename('daily_kwh')
weekly_agg = df.groupby('week')['USAGE_KWH'].sum().rename('weekly_kwh')
monthly_agg = df.groupby('month')['USAGE_KWH'].sum().rename('monthly_kwh')
# Merge back using map
df['hourly_kwh'] = df['hour'].map(hourly_agg)
df['daily_kwh'] = df['day'].map(daily_agg)
df['weekly_kwh'] = df['week'].map(weekly_agg)
df['monthly_kwh'] = df['month'].map(monthly_agg)

In [41]:
# summary_df = pd.DataFrame({
#     'hourly_kwh': [0.51, 0.48, 0.62],
#     'daily_kwh': [18.7, 19.2, 17.9],
#     'weekly_kwh': [125.4, 130.1, 122.8],
#     'monthly_kwh': [567.2, 571.0, 563.5]
# })
# Set datetime as index (if not already)
# summary_df.index = pd.to_datetime(datetime)
# summary_df.rename_axis('datetime', inplace=True)

                       # Reset index to make datetime a column
# summary_df = summary_df.reset_index()
# Extract date and time
summary_df['time'] = summary_df['datetime'].dt.strftime('%H:%M')
summary_df['date'] = summary_df['datetime'].dt.strftime('%Y-%m-%d')


In [42]:
summary_df

,datetime,hourly_kwh,daily_kwh,weekly_kwh,monthly_kwh,time,date
0,2025-11-30 00:00:00,1.277,57.087,57.087,57.087,00:00,2025-11-30
1,2025-11-30 01:00:00,1.200,NaN,NaN,NaN,01:00,2025-11-30
2,2025-11-30 02:00:00,1.177,NaN,NaN,NaN,02:00,2025-11-30
3,2025-11-30 03:00:00,1.233,NaN,NaN,NaN,03:00,2025-11-30
4,2025-11-30 04:00:00,1.366,NaN,NaN,NaN,04:00,2025-11-30
...,...,...,...,...,...,...,...
717,2025-12-29 21:00:00,4.097,NaN,NaN,NaN,21:00,2025-12-29
718,2025-12-29 22:00:00,4.147,NaN,NaN,NaN,22:00,2025-12-29
719,2025-12-29 23:00:00,3.737,NaN,NaN,NaN,23:00,2025-12-29
720,2025-12-31 00:00:00,NaN,NaN,NaN,1764.535,00:00,2025-12-31


In [49]:
peak = ['18:00','19:00','20:00','21:00']


In [66]:
def is_month_end(date_input):
    # accepts date_input as datetime.date, datetime.datetime, or "YYYY-MM-DD" str
    if isinstance(date_input, str):
        date = datetime.strptime(date_input, "%Y-%m-%d").date()
    elif isinstance(date_input, datetime):
        date = date_input.date()
    else:
        date = date_input  # assume datetime.date
    last_day = calendar.monthrange(date.year, date.month)[1]
    return date.day == last_day


In [74]:
df = summary_df
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['is_month_end']= df['datetime'].dt.is_month_end

In [80]:
df[df['is_month_end'] == True][df['monthly_kwh'] >0]

/var/folders/8g/jn8cz2bj1hg4ydc3f57xd6m40000gn/T/ipykernel_30271/1485008455.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['is_month_end'] == True][df['monthly_kwh'] >0]


,datetime,hourly_kwh,daily_kwh,weekly_kwh,monthly_kwh,time,date,hour,is_peak,peak_kwh,...,day,year,isMonthEnd,is_month_end,daily_peak_kwh,daily_non_peak_kwh,weekly_peak_kwh,weekly_non_peak_kwh,monthly_peak_kwh,monthly_non_peak_kwh
0,2025-11-30,1.277,57.087,57.087,57.087,00:00,2025-11-30,0,False,0.0,...,30,2025,True,True,8.643,48.444,0.0,0.0,8.643,48.444
720,2025-12-31,0.000,0.000,0.000,1764.535,00:00,2025-12-31,0,False,0.0,...,31,2025,True,True,0.000,0.000,0.0,0.0,279.090,1485.445


In [79]:
# Ensure date and time are properly parsed
# df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
# Extract hour from time
# df['hour'] = df['datetime'].dt.hour
# Define peak hours
peak_hours = [18, 19, 20, 21]  # Convert to integers for comparison
# Create peak flag
df['is_peak'] = df['hour'].isin(peak_hours)
# Separate peak and non-peak usage
df['peak_kwh'] = df['hourly_kwh'].where(df['is_peak'], 0)
df['non_peak_kwh'] = df['hourly_kwh'].where(~df['is_peak'], 0)
# Set datetime as index for resampling
df.set_index('datetime', inplace=True)
# Daily sums
df_daily = df.resample('D').agg({
    'peak_kwh': 'sum',
    'non_peak_kwh': 'sum'
}).rename(columns={'peak_kwh': 'daily_peak_kwh', 'non_peak_kwh': 'daily_non_peak_kwh'})
# Weekly sums
df_weekly = df.resample('W-MON').agg({
    'peak_kwh': 'sum',
    'non_peak_kwh': 'sum'
}).rename(columns={'peak_kwh': 'weekly_peak_kwh', 'non_peak_kwh': 'weekly_non_peak_kwh'})
# Monthly sums
df_monthly = df.resample('ME').agg({
    'peak_kwh': 'sum',
    'non_peak_kwh': 'sum'
}).rename(columns={'peak_kwh': 'monthly_peak_kwh', 'non_peak_kwh': 'monthly_non_peak_kwh'})
# Merge back into original DataFrame using the index
df.reset_index(inplace=True)
df = pd.merge(df, df_daily[['daily_peak_kwh', 'daily_non_peak_kwh']], 
              left_on='datetime', right_index=True, how='left')
df = pd.merge(df, df_weekly[['weekly_peak_kwh', 'weekly_non_peak_kwh']], 
              left_on='datetime', right_index=True, how='left')
df = pd.merge(df, df_monthly[['monthly_peak_kwh', 'monthly_non_peak_kwh']], 
              left_on='datetime', right_index=True, how='left')
# Fill NaNs with 0 (for first few days/weeks/months with no data)
df.fillna(0, inplace=True)
# Optional: Drop temporary columns if needed
# df.drop(columns=['hour', 'is_peak', 'datetime'], inplace=True)

In [56]:
df

,datetime,hourly_kwh,daily_kwh,weekly_kwh,monthly_kwh,time,date,hour,is_peak,peak_kwh,non_peak_kwh,daily_peak_kwh,daily_non_peak_kwh,weekly_peak_kwh,weekly_non_peak_kwh,monthly_peak_kwh,monthly_non_peak_kwh
0,2025-11-30 00:00:00,1.277,57.087,57.087,57.087,00:00,2025-11-30,0,False,0.000,1.277,8.643,48.444,0.0,0.0,8.643,48.444
1,2025-11-30 01:00:00,1.200,0.000,0.000,0.000,01:00,2025-11-30,1,False,0.000,1.200,0.000,0.000,0.0,0.0,0.000,0.000
2,2025-11-30 02:00:00,1.177,0.000,0.000,0.000,02:00,2025-11-30,2,False,0.000,1.177,0.000,0.000,0.0,0.0,0.000,0.000
3,2025-11-30 03:00:00,1.233,0.000,0.000,0.000,03:00,2025-11-30,3,False,0.000,1.233,0.000,0.000,0.0,0.0,0.000,0.000
4,2025-11-30 04:00:00,1.366,0.000,0.000,0.000,04:00,2025-11-30,4,False,0.000,1.366,0.000,0.000,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2025-12-29 21:00:00,4.097,0.000,0.000,0.000,21:00,2025-12-29,21,True,4.097,0.000,0.000,0.000,0.0,0.0,0.000,0.000
718,2025-12-29 22:00:00,4.147,0.000,0.000,0.000,22:00,2025-12-29,22,False,0.000,4.147,0.000,0.000,0.0,0.0,0.000,0.000
719,2025-12-29 23:00:00,3.737,0.000,0.000,0.000,23:00,2025-12-29,23,False,0.000,3.737,0.000,0.000,0.0,0.0,0.000,0.000
720,2025-12-31 00:00:00,0.000,0.000,0.000,1764.535,00:00,2025-12-31,0,False,0.000,0.000,0.000,0.000,0.0,0.0,279.090,1485.445


In [81]:
df.to_csv('electricity_usage_with_peaks.csv', index=False)


In [ ]:
df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format=' %H:%M').dt.hour


In [ ]:
df

In [58]:
df_monthly

,monthly_peak_kwh,monthly_non_peak_kwh
datetime,,
2025-11-30,8.643,48.444
2025-12-31,279.090,1485.445
2026-01-31,0.000,0.000
